In [1]:
from PIL import Image
import os
import shutil
import numpy as np
# import cv2
import re
import pickle
import random
import time
# from tqdm import tqdm
import pandas as pd 
from glob import glob
from xml.etree.ElementTree import parse
# from pascal_voc_writer import Writer
import matplotlib.pyplot as plt

import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.auto import tqdm


#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

#sklearn
from sklearn.model_selection import StratifiedKFold

#CV
import cv2

################# DETR FUCNTIONS FOR LOSS######################## 
import sys
sys.path.extend(['./detr/'])

from models.matcher import HungarianMatcher
from models.detr import SetCriterion
################################################################

#Albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

#Glob
from glob import glob

import sys
sys.path.extend(['./util/'])
from data_parser import *
from image_prepro import *
# from torchvision import transforms

from visualize import *

In [2]:
df = pd.read_csv('df.csv')
df.head()

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path
0,20200623_100015.jpg,4032,1960,1051,2061,124,117,Suamous cell carcinoma,Carcinoma,1,/home/beomgon/Object_Detection/Dataset/SS/06/2...
1,20200420_100401.jpg,4032,1960,870,1945,71,79,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
2,20200420_100401.jpg,4032,1960,947,2400,73,84,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
3,20200420_100401.jpg,4032,1960,845,2209,67,98,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
4,20200420_100401.jpg,4032,1960,811,2056,55,88,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...


In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
n_folds = 5
seed = 42
num_classes = 6
num_queries = 100
null_class_coef = 0.5
BATCH_SIZE = 2
LR = 2e-5
EPOCHS = 2

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [6]:
def get_data(img_id):
    if img_id not in df_data.groups:
        return dict(image_id=img_id, source='', boxes=list())
    
    data  = df_data.get_group(img_id)
    labels = data.label.values
#     source = data.source.values
#     source = np.unique(data.source.values)
#     assert len(source)==1, 'corrupted data: %s image_id has many sources: %s' %(img_id,source)
#     source=source[0]
    boxes = data[['x_min','y_min','box_width','box_height']].values
#     boxes = list(data.bbox)
    label_id = data.label_id.values
    return dict(image_id = img_id, labels=labels, label_id=label_id, boxes = boxes)

df_data = df.groupby('path')
images = list(set(df.path))
image_list = [get_data(img_id) for img_id in images]

In [16]:
df.head()

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path
0,20200623_100015.jpg,4032,1960,1051,2061,124,117,Suamous cell carcinoma,Carcinoma,1,/home/beomgon/Object_Detection/Dataset/SS/06/2...
1,20200420_100401.jpg,4032,1960,870,1945,71,79,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
2,20200420_100401.jpg,4032,1960,947,2400,73,84,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
3,20200420_100401.jpg,4032,1960,845,2209,67,98,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
4,20200420_100401.jpg,4032,1960,811,2056,55,88,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...


In [24]:
# # Creating Folds
# skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)

# df_folds = df[['path']].copy()
# df_folds.loc[:, 'bbox_count'] = 1
# df_folds = df_folds.groupby('path').count()
# df_folds.loc[:, 'fold'] = 0
# (df_folds)
# for fold_number, (train_index, val_index) in enumerate(len(df_folds)):
#     df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number


,bbox_count,fold
path,,
/home/beomgon/Object_Detection/Dataset/SS/00/20191025_100424.jpg,2,0
/home/beomgon/Object_Detection/Dataset/SS/00/20191025_100442.jpg,1,0
/home/beomgon/Object_Detection/Dataset/SS/00/20191025_100448.jpg,1,0
/home/beomgon/Object_Detection/Dataset/SS/00/20191025_100453.jpg,1,0
/home/beomgon/Object_Detection/Dataset/SS/00/20191025_100520.jpg,1,0
...,...,...
/home/beomgon/Object_Detection/Dataset/SS2/09(201117)/20201117_092737.jpg,1,0
/home/beomgon/Object_Detection/Dataset/SS2/09(201117)/20201117_092738.jpg,1,0
/home/beomgon/Object_Detection/Dataset/SS2/09(201117)/20201117_095706.jpg,1,0


In [26]:
len(df_folds)

18824

In [ ]:
# df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
# df_folds.loc[:, 'stratify_group'] = np.char.add(
#     df_folds['source'].values.astype(str),
#     df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
# )
# df_folds.loc[:, 'fold'] = 0

# for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
#     df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

In [19]:
transforms = A.Compose([
#     A.RandomCrop(width=450, height=450),
#     A.HorizontalFlip(p=1),
    A.CenterCrop(1560,1560, True,1),
#     A.RandomCrop(height=800, width=800, p=0.5),
#     A.Resize(height=512, width=512, p=1),
    A.Resize(height=256, width=256, p=1),
    
    A.OneOf([
    A.HorizontalFlip(p=0.9),
    A.VerticalFlip(p=0.9),
    A.RandomRotate90(p=0.9),        
    ], p=1),

    A.pytorch.ToTensor(),
# ], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.5))
], p=1.0, bbox_params=A.BboxParams(format='coco', min_area=0, min_visibility=0.5, label_fields=['labels'])) 

In [8]:

class WheatDataset(Dataset):
    def __init__(self,image_list,transforms=None):
        self.images = image_list
        self.transforms = transforms
        self.img_ids = {x['image_id']:i for i,x in enumerate(image_list)}
        
    def get_indices(self,img_ids):
        return [self.img_ids[x] for x in img_ids]
        
    def __len__(self) -> int:
        return len(self.images)
    
    def __getitem__(self,index):
        record = self.images[index]
        image_id = record['image_id']
#         print(image_id)

        image = cv2.imread(image_id, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = cv2.flip(image, 1)
        image /= 255.0
        
        # DETR takes in data in coco format 
        boxes = record['boxes'] 
#         print('boxes', boxes)
        
#         labels =  np.zeros(len(boxes), dtype=np.int32)
        labels = record['label_id']
#         labels = np.array(labels, dtype=np.int32)
#         print('label', type(labels))
#         print(labels)
#         print(image.shape)

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transforms(**sample)
            image  = sample['image']
            boxes  = sample['bboxes']
            labels = sample['labels']

        h,w, _ = image.shape
#         print('h', h)
#         print('w',w)
#         print(sample['bboxes'])
        boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
#         print('boxes', boxes)
        ## detr uses center_x,center_y,width,height !!
        if len(boxes)>0:
            boxes = np.array(boxes)
            boxes[:,2:] /= 2
            boxes[:,:2] += boxes[:,2:]
        else:
            boxes = np.zeros((0,4))
    
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels, dtype=torch.uint8)
        target['image_id'] = torch.tensor([index])
        
        return image, target, image_id

In [9]:
# train_ds = WheatDataset(image_list,transforms)

In [10]:
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

In [12]:
'''
code taken from github repo detr , 'code present in engine.py'
'''

# matcher = HungarianMatcher()
matcher = HungarianMatcher(cost_class=num_classes)

weight_dict = weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}

losses = ['labels', 'boxes', 'cardinality']

In [ ]:
def train_fn(data_loader,model,criterion,optimizer,device,scheduler,epoch):
    model.train()
    criterion.train()
    
    summary_loss = AverageMeter()
    
    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for step, (images, targets, image_ids) in enumerate(tk0):
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        

        output = model(images)
        
        loss_dict = criterion(output, targets)
        weight_dict = criterion.weight_dict
        
        losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        
        optimizer.zero_grad()

        losses.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        
        summary_loss.update(losses.item(),BATCH_SIZE)
        tk0.set_postfix(loss=summary_loss.avg)
        
    return summary_loss

In [13]:
def eval_fn(data_loader, model,criterion, device):
    model.eval()
    criterion.eval()
    summary_loss = AverageMeter()
    
    with torch.no_grad():
        
        tk0 = tqdm(data_loader, total=len(data_loader))
        for step, (images, targets, image_ids) in enumerate(tk0):
            
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            output = model(images)
        
            loss_dict = criterion(output, targets)
            weight_dict = criterion.weight_dict
        
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
            
            summary_loss.update(losses.item(),BATCH_SIZE)
            tk0.set_postfix(loss=summary_loss.avg)
    
    return summary_loss

In [14]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
def run(fold):
    
    df_train = df_folds[df_folds['fold'] != fold]
    df_valid = df_folds[df_folds['fold'] == fold]
    
    train_dataset = WheatDataset(
    image_ids=df_train.index.values,
    dataframe=marking,
    transforms=get_train_transforms()
    )

    valid_dataset = WheatDataset(
    image_ids=df_valid.index.values,
    dataframe=marking,
    transforms=get_valid_transforms()
    )
    
    train_data_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
    )

    valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
    )
    
    device = torch.device('cuda')
    model = DETRModel(num_classes=num_classes,num_queries=num_queries)
    model = model.to(device)
    criterion = SetCriterion(num_classes-1, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
    criterion = criterion.to(device)
    

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    
    best_loss = 10**5
    for epoch in range(EPOCHS):
        train_loss = train_fn(train_data_loader, model,criterion, optimizer,device,scheduler=None,epoch=epoch)
        valid_loss = eval_fn(valid_data_loader, model,criterion, device)
        
        print('|EPOCH {}| TRAIN_LOSS {}| VALID_LOSS {}|'.format(epoch+1,train_loss.avg,valid_loss.avg))
        
        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            print('Best model found for Fold {} in Epoch {}........Saving Model'.format(fold,epoch+1))
            torch.save(model.state_dict(), f'detr_best_{fold}.pth')